In [1]:
import numpy as np
import pandas as pd

In [9]:
train_data = pd.read_csv('train_fake_news.csv', index_col = 'id')
test_data = pd.read_csv('test_fake_news.csv', index_col = 'id')

In [26]:
# Train data
train_data = train_data.rename(columns = {'title': 'Title', 'author': 'Author', 'text': 'Text', 'label': 'Label'})
idx = train_data.index
train_data.index = idx.rename('ID')

lenght = [len(str(train_data.iloc[i]['Text'])) for i in range(len(train_data))]
train_data['Lenght'] = lenght
train_data = train_data[train_data['Lenght'] > 50]

# Test data
test_data = test_data.rename(columns = {'title': 'Title', 'author': 'Author', 'text': 'Text', 'label': 'Label'})
idx = test_data.index
test_data.index = idx.rename('ID')

lenght = [len(str(test_data.iloc[i]['Text'])) for i in range(len(test_data))]
test_data['Lenght'] = lenght
test_data = test_data[test_data['Lenght'] > 50]


In [27]:
train_data

,Title,Author,Text,Label,Lenght
ID,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,1814
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,6059
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,4799


In [4]:
# Lenght of the maximal lenght text

max([len(str(train_data.iloc[i]['Text'])) for i in range(20800)])

142961

In [28]:
# Train data
train_data = train_data.drop(['Title', 'Author'], axis = 1)
train_data = train_data.dropna(axis = 0)
train_data.isnull().sum()

# Test data
test_data = test_data.drop(['Title', 'Author'], axis = 1)
test_data = test_data.dropna(axis = 0)
test_data.isnull().sum()

Text      0
Lenght    0
dtype: int64

In [29]:
train_data

,Text,Label,Lenght
ID,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,Ever get the feeling your life circles the rou...,0,4160
2,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Print \nAn Iranian woman has been sentenced to...,1,938
...,...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0,1814
20796,When the Green Bay Packers lost to the Washing...,0,6059
20797,The Macy’s of today grew from the union of sev...,0,4799


In [30]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [49]:
tokenizer = Tokenizer(num_words = 2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(train_data['Text'][:4000])
X = tokenizer.texts_to_sequences(train_data['Text'][:4000])

In [50]:
X = pad_sequences(X, maxlen = 2000, padding = 'pre')
y = train_data['Label'][:4000].values

In [51]:
print(X.shape)
print(y.shape)

(4000, 2000)
(4000,)


In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [53]:
# Building the LSTM model

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Embedding

In [54]:
model = Sequential()

model.add(Embedding(input_dim = 2000, output_dim = 100))

model.add(LSTM(units = 100, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 100, return_sequences = False))
model.add(Dropout(0.2))

model.add(Dense(100, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [55]:
training = model.fit(X_train, y_train, epochs = 1, batch_size = 64, verbose = 1)

C:\Users\User\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
2800/2800 [==============================] - 1662s 593ms/step - loss: 0.5238 - accuracy: 0.7411


In [57]:
prediction = model.predict(X_test)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

[[435 155]
 [ 44 566]]
              precision    recall  f1-score   support

           0       0.91      0.74      0.81       590
           1       0.79      0.93      0.85       610

    accuracy                           0.83      1200
   macro avg       0.85      0.83      0.83      1200
weighted avg       0.85      0.83      0.83      1200



In [63]:
prediction = prediction > 0.5
prediction = prediction.ravel()
prediction

array([ True,  True,  True, ...,  True,  True, False])

In [65]:
# Saving the model

model.save('fake_model.h5')

In [67]:
# Preparing the test data

tokenizer = Tokenizer(num_words = 2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(test_data['Text'])
test_X = tokenizer.texts_to_sequences(test_data['Text'])
test_X = pad_sequences(test_X, maxlen = 2000, padding = 'pre')

In [68]:
pred_test = model.predict(test_X)

In [69]:
pred_test = pred_test > 0.5
pred_test = pred_test.ravel()

In [74]:
result = pd.DataFrame({'ID' : test_data.index, 'Label' : pred_test})
# result.to_csv('fake_news.csv', index = False)

,ID,Label
0,20800,False
1,20801,True
2,20802,False
3,20803,False
4,20804,True
...,...,...
5123,25995,False
5124,25996,False
5125,25997,False
5126,25998,False
